[Advent of Code - Day 15](https://adventofcode.com/2019/day/15)

# Import input

In [ ]:
import sys
from collections import defaultdict

sys.path.insert(0, "../")
from utils import aoc_input as inp

INPUT = inp.download_input(year="2019", day="15")[0]

In [ ]:
INPUT[:5]

# Solulu

In [ ]:
def solve(intcode: list, direction:int, pointer: int, relative_base:int):
    while True:
        opcode, modes, total_values = decode(intcode[pointer])
        if opcode == 99:
            break

        elif opcode == 9:
            relative_base += parameter_mode(modes[0], pointer, 1, relative_base)

        elif opcode == 4:
            output_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            output_value = intcode[output_index]
            
            return (output_value, direction, pointer+ total_values, relative_base)

        elif opcode == 3:
            input_value = direction
            input_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            intcode[input_index] = input_value

        elif opcode in [1, 2, 7, 8]:
            nums = []
            for i, mode in enumerate(modes[:-1], start=1):
                num = parameter_mode(mode, pointer, i, relative_base)
                nums.append(num)
            if opcode == 1:
                value = nums[0] + nums[1]
            elif opcode == 2:
                value = nums[0] * nums[1]
            elif opcode == 7:
                value = nums[0] < nums[1]
            elif opcode == 8:
                value = nums[0] == nums[1]
            value_index = parameter_mode(
                modes[-1], pointer, 3, relative_base, index=True
            )
            intcode[value_index] = value

        if opcode == 5 and parameter_mode(modes[0], pointer, 1, relative_base) != 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        elif opcode == 6 and parameter_mode(modes[0], pointer, 1, relative_base) == 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        else:
            pointer += total_values

    return steps


def parameter_mode(mode, pointer, param, relative_base, index: bool = False) -> int:
    if mode == 0:
        num_index = intcode[pointer + param]
    elif mode == 1:
        num_index = pointer + param
    elif mode == 2:
        num_index = intcode[pointer + param] + relative_base

    return num_index if index else intcode[num_index]


def decode(instruction):
    opcode = instruction % 100
    modes = list()
    total_values = 1

    mode1 = (instruction // 100) % 10
    modes.append(mode1)
    if opcode not in (3, 4, 9):
        mode2 = instruction // 1000 % 10
        modes.append(mode2)
        if opcode not in (5, 6):
            mode3 = instruction // 10000
            modes.append(mode3)

    total_values += len(modes)

    return opcode, modes, total_values


def _reset_memory(inp=INPUT):
    intcode = list(map(int, INPUT.split(",")))
    d = defaultdict(int)
    for address, value in enumerate(intcode):
        d[address] = value
    return d

## Part 1

In [ ]:
intcode = _reset_memory(INPUT)
solve(intcode, 3, 249, 0)

In [ ]:
intcode = _reset_memory(INPUT)

queue = [[(0, 0), 0, 0,0]]
visited = set()
directions = {1: (0, -1), 2: (0, +1), 3: (-1, 0), 4: (+1, 0)}
input_values = [1, 2, 3, 4]

output = -1
pointer, relative_base = 0,0

while output < 2:
    (x,y), steps, pointer, relative_base= queue.pop(0)
    if (x,y) in visited:
        continue
    visited.add((x,y))

    for direction in input_values:
        output, direction, p, rel_base = solve(intcode, direction, pointer, relative_base)
        if output > 0:
            xx, yy = directions[direction]
            queue.append([(x+xx, y+yy), steps + 1, p, rel_base])
    
steps

## Part 2